In [28]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import datetime
import os
from astropy.timeseries import LombScargle 
from jdcal import gcal2jd

ModuleNotFoundError: No module named 'jdcal'

In [12]:
col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]

for i in range(1, 33):
    col_names.append(f"Sp{i}")

folder_path = 'Data'
file_name = 'StB LL 0508-0517-2024.txt'
file_path = os.path.join(folder_path, file_name)

df = pd.read_csv(file_path, names=col_names, sep='\s+', header=None)
df = df.set_index('Index')
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
df = df[df["MonStatus"] == 1]


month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7}
df['DateM'] = df['DateM'].str[:3].map(month_map)
df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                     month=df['Date'].dt.month,
                                     day=df['Date'].dt.day,
                                     hour=df['Time'].dt.hour,
                                     minute=df['Time'].dt.minute,
                                     second=df['Time'].dt.second))

df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}

df.insert(0, 'Day', df['Time'].dt.day.map(day_map))

spiders = []

for i in range(1, 33):
    if df[f"Sp{i}"].sum() > 0:
        spiders.append(i)
    if not i in spiders:
        df = df.drop([f"Sp{i}"], axis=1)


group_name = file_name.split(' ', 1)[0]

path = group_name + "_raster_plots"

display(df)

,Day,Time,Light,Sp1,Sp2,Sp3,Sp4,Sp5,Sp6,Sp7,...,Sp12,Sp13,Sp14,Sp17,Sp18,Sp19,Sp20,Sp21,Sp22,Sp23
Index,,,,,,,,,,,,,,,,,,,,,
99,1,2024-05-08 10:33:00,1,0,0,0,0,0,0,0,...,0,3,0,1,0,1,0,3,3,0
100,1,2024-05-08 10:34:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
101,1,2024-05-08 10:35:00,1,0,0,0,0,0,0,7,...,0,0,0,0,0,6,0,0,0,0
102,1,2024-05-08 10:36:00,1,0,4,0,0,0,0,1,...,0,0,0,0,0,4,0,0,0,0
103,1,2024-05-08 10:37:00,1,0,2,0,0,0,0,4,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13092,10,2024-05-17 11:06:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13093,10,2024-05-17 11:07:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13094,10,2024-05-17 11:08:00,1,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Raster plot code

In [14]:
def raster_plot(sp, fig):
    for j in range(1, len(df['Day'].unique()) + 1):
        curr_df = df[df['Day']==j]
        
        curr_df_avg = curr_df.groupby(curr_df['Time'].dt.hour)[sp].mean()
    
        ax = fig.add_subplot(len(df['Day'].unique()), 1, j)
        ax.set_ylabel(f"Day {j}", rotation="horizontal", va="center", ha="right", fontsize=8)

        
        for k in range(0, len(curr_df) -1):
            start_time = curr_df['Time'].iloc[k]
            end_time = curr_df['Time'].iloc[k + 1]
            if curr_df['Light'].iloc[k] == 1:
                ax.axvspan(start_time.hour + start_time.minute / 60, end_time.hour + end_time.minute / 60, color='yellow', alpha=0.3)

        curr_df_avg.plot.bar(y = [sp], ax=ax)
        
        ax.set_ylim(0, 1)
        ax.legend().remove()
        ax.tick_params(left=False, bottom=False)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_xlim(-0.5, 23.5)
        ax.set_xlabel("")
        
        if j == 1:
            ax.set_title(sp)
            #ax.text(0.5, 1.5, x, fontsize=12, horizontalalignment='center', transform=ax.transAxes)
            
        if j == len(df["Day"].unique()):
            ax.set_xticks(np.arange(0, 25, 2))
            ax.set_xticklabels(np.arange(0, 25, 2), rotation = 'horizontal', fontsize = 7)
            ax.set_xlabel('Time (hours)')
    
    file_path = os.path.join(path, f"{sp}_raster_plot.png")
    plt.savefig(file_path)
    plt.close()
    #plt.show()

In [16]:
if not os.path.exists(path):
    os.makedirs(path)


for i in spiders:
    x = f"Sp{i}"
    fig = plt.figure(figsize=(4, 6))
    raster_plot(x, fig)


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fc0821212d0> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

# Lomb-Scargle Code

In [20]:


def datetime_to_jd(dt):
    year, month, day = dt.year, dt.month, dt.day
    hour, minute, second = dt.hour, dt.minute, dt.second
    _, jd = gcal2jd(year, month, day)
    jd += (hour + minute / 60 + second / 3600) / 24
    return jd


def period_LS(df, sp, first_day, last_day):
    
    curr_df = df[(df["Day"] >= first_day) & (df["Day"] <= last_day)]

    activity = np.array(curr_df[sp])

    time = curr_df['Time'].apply(datetime_to_jd)

    ls = LombScargle(time, activity)
    freq = np.linspace(0.1, 2, len(activity))
    power = ls.power(freq)

    max_power = np.max(power)
    max_freq = freq[np.argmax(power)]
    period = 1 / max_freq 
    period *= 24
    print(f"The Lomb-Scargle approximation of the period is {period} hours.")

    plt.figure(figsize=(10, 6))
    plt.plot(freq, power)
    plt.scatter(max_freq, max_power, s=75, c="red")
    plt.xlabel('Frequency (cycles/day)')
    plt.ylabel('Power')
    plt.title(f'Lomb-Scargle Periodogram for {sp}')
    plt.show()




In [22]:
period_LS(df, 'Sp17', 1, 7)
period_LS(df, 'Sp17', 8, 15)

NameError: name 'gcal2jd' is not defined